In [1]:
# Script to post-process high-resolution WRF model output.

# Major tasks include computing the following for selected variables:
#   1. domain-averages to produce time series
#   2. vertical integrals
#   3. pressure-level vertical interpolation

In [2]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, vinterp, ALL_TIMES
import xarray as xr
from read_wrf_ideal import *
from post_proc_functions import *
import os

In [3]:
# Special 2D variables
do_2d_special = True
# Basic 3D variables
do_3d_vars = False
# Special 3D variables
do_3d_special = False

In [4]:
#### Directories and model output specs

test_process = "ctl"

# wrfdir = "/glade/campaign/univ/uokl0049/ideal/largedom2/"+test_process+"/"
wrfdir = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/wrf-ideal/largedom2/"+test_process+"/"
outdir = wrfdir+"post_proc/"
os.makedirs(outdir, exist_ok=True)

# # Get WRF file list, dimensions
wrftag = "wrfout_d01"
wrffiles = get_wrf_file_list(wrfdir, wrftag)
lat, lon, nx1, nx2, nz, npd = wrf_dims(wrffiles[0])
nfiles = len(wrffiles)

# New vertical dimension for pressure levels
dp = 25 # hPa
pres = np.arange(1000, 25, -dp)
nznew = len(pres)

# Get variable lists
vars3d = var_list_3d()

In [28]:
if do_2d_special:

    # Read in variable from WRF files
    # for ifile in range(nfiles):

    # grav = 9.81 # m/s^2
    # hght_bs = wrf_var_read0(wrffiles[0],'PHB')/grav # m
    # print(hght_bs.shape)

    for ifile in range(1):

        # Read in hydrostatic pressure to get dp for integral
        # p_hyd = wrf_var_read(wrffiles[ifile],'P_HYD') # Pa
        # p_hyd = np.ma.masked_where((p_hyd < 100e2), p_hyd, copy=False) # Mask out levels above 100 hPa
        # dp = np.gradient(p_hyd, axis=0, edge_order=1) # [Pa] Uses second order centered differencing

        # Use staggered height instead
        # hght_pert = wrf_var_read(wrffiles[ifile],'PH')/grav # m
        
        dset = Dataset(wrffiles[0])
        hght = getvar(dset, "zstag", units='m', timeidx=ALL_TIMES)#, cache=cache)
        # Also need density
        # tv = getvar(dset, "tv", units='K', timeidx=ALL_TIMES)#, cache=cache)
        qv = getvar(dset, "QVAPOR", timeidx=ALL_TIMES)#, cache=cache)
        tmpk = getvar(dset, "tk", timeidx=ALL_TIMES)#, cache=cache)
        pwrf = getvar(dset, "p", units='Pa', timeidx=ALL_TIMES)#, cache=cache)
        # rho = density_dry(tv, pwrf)
        rho = density_moist(tmpk, qv, pwrf)
        dset.close()

        # Get dz
        dz = np.zeros(qv.shape)
        for iz in range(nz):
            dz[:,iz] = hght[:,iz+1] - hght[:,iz]

        # Read in variables

        # rainrate
        var = wrf_var_read(wrffiles[ifile], 'RAINNC')
        # var = getvar(dset, "RAINNC", timeidx=ALL_TIMES)#, cache=cache)
        if ifile == 0:
            rainnc_all = var
        else:
            rainnc_all = np.concatenate((rainnc_all, var), axis=0)
        # pclass
        var = wrf_pclass(wrffiles[ifile], rho, dz)
        if ifile == 0:
            pclass_all = var
        else:
            pclass_all = np.concatenate((pclass_all, var), axis=0)
        # pw
        # var = wrf_pw(wrffiles[ifile], rho, dz)
        var = vert_int(qv, rho, dz)
        if ifile == 0:
            pw_all = var
        else:
            pw_all = np.concatenate((pw_all, var), axis=0)
        # pw_sat
        # var = wrf_pw_sat(wrffiles[ifile], tmpk, pwrf, rho, dz)
        var = vert_int(rv_saturation(tmpk, pwrf), rho, dz)
        if ifile == 0:
            pw_sat_all = var
        else:
            pw_sat_all = np.concatenate((pw_sat_all, var), axis=0)

    # Calculate rain rate as centered difference
    nt_all = rainnc_all.shape[0]
    rainrate = np.full((nt_all, nx1, nx2), np.nan)
    rainrate[0] = 0
    rainrate[1:-1] = (rainnc_all[2:] - rainnc_all[:-2])*0.5
    rainrate *= npd # mm/time step --> mm/day

    # Write out the variables
    name='pclass'
    description, units, dim_set = get_metadata('pclass', nt_all, nz, nx1, nx2)
    write_ncfile(outdir+name+'.nc', qv[:,0], pclass_all, name, description, units)
    name='rainrate'
    description, units, dim_set = get_metadata(name, nt_all, nz, nx1, nx2)
    write_ncfile(outdir+name+'.nc', qv[:,0], rainrate, name, description, units)
    name='pw'
    description, units, dim_set = get_metadata('pw', nt_all, nz, nx1, nx2)
    write_ncfile(outdir+name+'.nc', qv[:,0], pw_all, name, description, units)
    name='pw_sat'
    description, units, dim_set = get_metadata('pw_sat', nt_all, nz, nx1, nx2)
    write_ncfile(outdir+name+'.nc', qv[:,0], pw_sat_all, name, description, units)

(48, 79, 266, 266)
(48, 79, 266, 266)
(48, 79, 266, 266)
(48, 79, 266, 266)
(48, 79, 266, 266)


In [8]:
if do_3d_vars:

    for ivar in vars3d[0:1]:

        varname_str = ivar.upper()

        # Read in variable from WRF files
        # for ifile in range(nfiles):
        print(ivar.upper())
        for ifile in range(1):
            wrfdset = Dataset(wrffiles[ifile])
            var = wrfdset.variables[ivar.upper()][...]
            wrfdset.close()
            if ifile == 0:
                var_all = var
            else:
                var_all = np.concatenate((var_all, var), axis=0)

In [9]:
# get_cache=True
# get_cache=False

# if get_cache:
#     cache = {
#         "PSFC": getvar(wrffil_read, "PSFC", timeidx=ALL_TIMES),
#         "QVAPOR": getvar(wrffil_read, "QVAPOR", timeidx=ALL_TIMES),
#         "terht": get_terrain(wrffil_read, ALL_TIMES, units="m"),
#         "tk": get_temp(wrffil_read, ALL_TIMES, units="k"),
#         "p": get_pressure(wrffil_read, ALL_TIMES, units="pa"),
#         "ght": get_height(wrffil_read, ALL_TIMES, msl=True, units="m")
#     }
# else:
#     cache=None

In [26]:
dset = Dataset(wrffiles[0])
qv = getvar(dset, "QVAPOR", timeidx=ALL_TIMES)#, cache=cache)
qv_interp = vinterp(dset, qv, 'p', pres, extrapolate=True, timeidx=ALL_TIMES)#, cache=cache)
dset.close()
qv_interp.shape

(48, 39, 266, 266)

In [28]:
# wrffil_read = Dataset(wrffiles[0])
# psfc = getvar(wrffil_read, "PSFC", timeidx=ALL_TIMES)#, cache=cache)
# p = getvar(wrffil_read, "pressure", timeidx=5)#, cache=cache)
# wrffil_read.close()
# p[0,:,:]

In [30]:
qv_interp[40,:,20,20]

<xarray.DataArray 'QVAPOR' (interp_level: 39)> Size: 156B
array([1.3966289e-02, 1.3660062e-02, 1.3513280e-02, 1.3256672e-02,
       1.1282385e-02, 9.4764894e-03, 8.8272905e-03, 8.3505306e-03,
       7.9362560e-03, 7.5409873e-03, 7.1207182e-03, 6.6800923e-03,
       6.2200688e-03, 5.7087368e-03, 5.1546991e-03, 4.6128090e-03,
       4.1116029e-03, 3.6190012e-03, 3.1075752e-03, 2.5775805e-03,
       2.0434333e-03, 1.5356819e-03, 1.1056715e-03, 7.6150120e-04,
       5.2808394e-04, 3.9898106e-04, 3.1992700e-04, 2.6116025e-04,
       2.1806285e-04, 1.6002244e-04, 8.9467889e-05, 4.4995009e-05,
       1.9985580e-05, 8.0299751e-06, 2.7746244e-06, 1.0129648e-06,
       3.7744110e-09, 2.4223578e-13, 3.0842972e-21], dtype=float32)
Coordinates:
    XLONG         float32 4B 0.0
    XLAT          float32 4B 10.0
    XTIME         float32 4B 1.2e+03
    Time          datetime64[ns] 8B 2007-06-01T20:00:00
  * interp_level  (interp_level) int64 312B 1000 975 950 925 ... 125 100 75 50
Attributes:
    FieldType:         104
    MemoryOrder:       XYZ
    description:       Water vapor mixing ratio
    units:             kg kg-1
    stagger:           
    coordinates:       XLONG XLAT XTIME
    projection:        RotatedLatLon(stand_lon=0.0, moad_cen_lat=0.0, truelat...
    vert_interp_type:  p

In [11]:
print('Time without cache: 1:23')
print('Time with cache: 2:23')

Time without cache: 1:32
Time with cache: 2:23


In [32]:
qv_interp

<xarray.DataArray 'QVAPOR' (Time: 48, interp_level: 39, south_north: 266,
                            west_east: 266)> Size: 530MB
array([[[[1.39092291e-02, 1.39092291e-02, 1.39092291e-02, ...,
          1.39092291e-02, 1.39092291e-02, 1.39092291e-02],
         [1.39092291e-02, 1.39092291e-02, 1.39092291e-02, ...,
          1.39092291e-02, 1.39092291e-02, 1.39092291e-02],
         [1.39092291e-02, 1.39092291e-02, 1.39092291e-02, ...,
          1.39092291e-02, 1.39092291e-02, 1.39092291e-02],
         ...,
         [1.39092291e-02, 1.39092291e-02, 1.39092291e-02, ...,
          1.39092291e-02, 1.39092291e-02, 1.39092291e-02],
         [1.39092291e-02, 1.39092291e-02, 1.39092291e-02, ...,
          1.39092291e-02, 1.39092291e-02, 1.39092291e-02],
         [1.39092291e-02, 1.39092291e-02, 1.39092291e-02, ...,
          1.39092291e-02, 1.39092291e-02, 1.39092291e-02]],

        [[1.36916609e-02, 1.36916609e-02, 1.36916600e-02, ...,
          1.36916609e-02, 1.36916609e-02, 1.36916600e-02],
         [1.36916609e-02, 1.36916600e-02, 1.36916600e-02, ...,
          1.36916609e-02, 1.36916600e-02, 1.36916609e-02],
         [1.36916600e-02, 1.36916600e-02, 1.36916600e-02, ...,
          1.36916609e-02, 1.36916600e-02, 1.36916609e-02],
...
          2.22877421e-13, 3.74802348e-13, 4.12277661e-13],
         [3.92395019e-13, 4.36261406e-13, 5.65298459e-13, ...,
          2.09121457e-13, 3.43425565e-13, 4.26597722e-13],
         [4.00614139e-13, 4.36407122e-13, 4.51029649e-13, ...,
          1.73042719e-13, 2.30537242e-13, 2.78453259e-13]],

        [[2.20674117e-21, 3.46571390e-20, 1.62719518e-15, ...,
          1.72145963e-20, 5.38283001e-20, 2.01550553e-21],
         [1.27487554e-15, 1.89696120e-15, 2.42762459e-15, ...,
          5.30130725e-21, 2.14034801e-21, 6.79911441e-16],
         [5.54546668e-15, 4.90431102e-15, 3.97609547e-15, ...,
          1.44425248e-15, 3.75248555e-15, 5.38954020e-15],
         ...,
         [8.89825715e-20, 5.33487680e-20, 9.96477194e-16, ...,
          5.66118049e-21, 1.03783187e-21, 1.04006259e-19],
         [1.14330281e-19, 5.89030920e-20, 2.70466620e-15, ...,
          2.26197724e-23, 1.22573451e-17, 4.59625815e-21],
         [2.93817381e-21, 9.58781041e-20, 2.33244032e-15, ...,
          1.65363131e-22, 3.31469625e-20, 1.56650853e-21]]]],
      dtype=float32)
Coordinates:
    XLONG         (south_north, west_east) float32 283kB 0.0 0.0 0.0 ... 0.0 0.0
    XLAT          (south_north, west_east) float32 283kB 10.0 10.0 ... 10.0 10.0
    XTIME         (Time) float32 192B 0.0 30.0 60.0 ... 1.38e+03 1.41e+03
  * Time          (Time) datetime64[ns] 384B 2007-06-01 ... 2007-06-01T23:30:00
  * interp_level  (interp_level) int64 312B 1000 975 950 925 ... 125 100 75 50
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:         104
    MemoryOrder:       XYZ
    description:       Water vapor mixing ratio
    units:             kg kg-1
    stagger:           
    coordinates:       XLONG XLAT XTIME
    projection:        RotatedLatLon(stand_lon=0.0, moad_cen_lat=0.0, truelat...
    vert_interp_type:  p

In [31]:
outfile = outdir+"qv_interp.nc"
qv_interp.to_netcdf(outfile, mode='w', format='NETCDF4')


TypeError: Invalid value for attr 'projection': RotatedLatLon(stand_lon=0.0, moad_cen_lat=0.0, truelat1=0.0, truelat2=0.0, pole_lat=0.0, pole_lon=0.0). For serialization to netCDF files, its value must be of one of the following types: str, Number, ndarray, number, list, tuple